# 1. 데이터 전처리

- 데이터 분리

In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
total_df = pd.read_csv("원주20200401_20200630.csv")
total_df = total_df.append(pd.read_csv("원주20200701_20200930.csv"))
total_df = total_df.append(pd.read_csv("원주20201001_20201231.csv"))
total_df = total_df.append(pd.read_csv("원주20210101_20210228.csv"))
total_df = total_df.append(pd.read_csv("원주20210301_20210326.csv"))

people = total_df['수용가번호'].drop_duplicates()
# people 수용가 번호 (고객 구분 변수) 로 customer 구분

- 사람별 분리

In [ ]:
from tqdm import tqdm
# 적산유량보정값은 누적값이므로 어느 시점부터 값이 생김
# 적산유량보정값이 없는 앞부분 제거

# sz : 총 데이터 열 개수
sz = []
for p in tqdm(people):
  df_someone = total_df[total_df['수용가번호']==p]
  
  for drop_index in range(df_someone.shape[0]):
    if not pd.isna(df_someone.loc[:, '적산유량보정값'].iloc[drop_index]):
      break
  df_someone = df_someone.iloc[drop_index:]
  sz.append(df_someone.shape[0])

  # 적산유량보정값 없는 부분 제거하고
  # 데이터가 너무 짧으면 사용하지 않음
  if df_someone.shape[0] < 2000:
    continue
   
  # 정상이라면 0, 정상이 아니라면 (누수, 고장 전부) 1
  df_someone['상태코드'] = (df_someone['상태코드'] != '정상').astype(int)
  t = 0.0
  nxt = 1
  features = ['미터배터리', '적산유량보정값', '구간유량보정값', '지침값오프셋', 'snr']
  new_df = df_someone.iloc[:1] # 새로 만들어지는 사람별 데이터
  receive_check = [1]
  receive_cnt = 0
  # 수신되지 않은 시간에 na를 채우고 또 1시간 단위에서 어긋난 데이터를 보간하여 채운다.
  for i in range(1, df_someone.shape[0]):
    t += df_someone.iloc[i]['시간간격']
    receive_cnt += 1
    while nxt <= t:
      if abs(nxt - t) < 0.02:
        new_df = new_df.append(df_someone.iloc[i:i+1])
        receive_check.append(receive_cnt)
        receive_cnt = 0
      else:
        before_row = new_df.iloc[-1]
        one_row = df_someone.iloc[i]
        if (nxt + 1 < t): # 긴 시간동안 수신되지 않으면 보간하지 않고 na를 채운다
          for f in features:
            one_row[f] = np.nan
        else:             # 1시간 간격을 어긋난 수준에서 비어있다면 보간하여 데이터를 채운다
          m = t - nxt + 1
          for f in features:
            one_row[f] = 1/m * one_row[f] + (t - nxt)/m * before_row[f]
          # 보간 후 구간 유량보정값을 올바르게 한번 더 수정한다.
          one_row['구간유량보정값'] = one_row['적산유량보정값']-before_row['적산유량보정값']
        receive_check.append(receive_cnt)
        receive_cnt = 0
        new_df = new_df.append(one_row)
      nxt += 1
  new_df.loc[:, '순번'] = list(range(0, new_df.shape[0]))
  re_ch = pd.DataFrame({'순번': new_df.loc[:, '순번'], '수신횟수':receive_check},)

  new_df = new_df.join(re_ch.set_index('순번'), on='순번')
  new_df.to_csv("사람별\\"+str(p)+".csv", sep=',', na_rep='NaN')

- 사람별 데이터 통합
- 기존 Data로부터 임의의 Feature 생성

In [ ]:
file_names = glob.glob("사람별/??????.csv")

total_df = None
df_cnt = 0

for f_name in file_names:
  df = pd.read_csv(f_name)

  # NaN 값이 많은 데이터 제거
  if df.count(axis=0)['적산유량보정값']/df.shape[0] <= 0.9:
    continue;
  
  # 선형 보간법 통해 Missing Value 보간
  df['적산유량보정값'] = df['적산유량보정값'].interpolate("linear", limit_area='inside')
  df.iloc[1:]['구간유량보정값'] = np.array(df.iloc[1:]['적산유량보정값']) - np.array(df.iloc[:-1]['적산유량보정값'])
  
  arrays = []
  # 레이블
  arrays.append(np.array(df.loc[:, '상태코드'].iloc[1:]).reshape([-1, 1]))

  # 유량
  arrays.append(np.array(df.loc[:, '구간유량보정값'].iloc[1:]).reshape([-1, 1]))

  # 한 달간 0 개수
  cnt = 0
  q = []
  zerocnt_month = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    if df.loc[:, '구간유량보정값'].iloc[i] == 0:
      q.append(1)
      cnt += 1
    else:
      q.append(0)
    zerocnt_month[i-1] = cnt
    if (len(q) >= 30*24):
      cnt -= q[0]
      q = q[1:]
  arrays.append(zerocnt_month)

  # 일주일간 0개수
  cnt = 0
  q = []
  zerocnt_week = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    if df.loc[:, '구간유량보정값'].iloc[i] == 0:
      q.append(1)
      cnt += 1
    else:
      q.append(0)
    zerocnt_week[i-1] = cnt
    if (len(q) >= 7*24):
      cnt -= q[0]
      q = q[1:]
  arrays.append(zerocnt_week)

  # 연속 0개수
  cnt = 0
  zerocnt = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    if df.loc[:, '구간유량보정값'].iloc[i] == 0:
      cnt += 1
    else:
      cnt = 0
    zerocnt[i-1] = cnt
  arrays.append(zerocnt)

  # diff_week
  diffweek = np.zeros([df.shape[0]-1, 1])
  q = [0 for i in range(2*7*24-1)]
  for i in range(1, df.shape[0]):
    q.append(df.loc[:, '구간유량보정값'].iloc[i])
    diffweek[i-1] = sum(q[7*24:])/(7*24) - sum(q[:7*24])/(7*24)
    if len(q) >= 14*24:
      q = q[1:]
  arrays.append(diffweek)

  # diff_day (최근 1주와 오늘의 사용량 차이)
  diffday = np.zeros([df.shape[0]-1, 1])
  q = [0 for i in range(8*24-1)]
  for i in range(1, df.shape[0]):
    q.append(df.loc[:, '구간유량보정값'].iloc[i])
    diffday[i-1] = sum(q[7*24:])/24 - sum(q[:7*24])/(7*24)
    if len(q) >= 8*24:
      q = q[1:]
  arrays.append(diffday)

  # diff_month (최근 한달과 오늘의 사용량 차이)
  diffmonth = np.zeros([df.shape[0]-1, 1])
  q = [0 for i in range(31*24-1)]
  for i in range(1, df.shape[0]):
    q.append(df.loc[:, '구간유량보정값'].iloc[i])
    diffmonth[i-1] = sum(q[30*24:])/24 - sum(q[:30*24])/(30*24)
    if len(q) >= 31*24:
      q = q[1:]
  arrays.append(diffmonth)

  # diff_month2 (최근 한달과 오늘의 사용량 차이, 크기보정)
  diffmonth2 = np.zeros([df.shape[0]-1, 1])
  q = [0 for i in range(31*24-1)]
  for i in range(1, df.shape[0]):
    q.append(df.loc[:, '구간유량보정값'].iloc[i])
    diffmonth2[i-1] = (sum(q[30*24:])/24 - sum(q[:30*24])/(30*24)) / (sum(q[:30*24])/(30*24)+0.01)
    if len(q) >= 31*24:
      q = q[1:]
  arrays.append(diffmonth2)

  # std_week (최근 1주 표준편차)
  std_week = np.zeros([df.shape[0]-1, 1])
  q = []
  for i in range(1, df.shape[0]):
    q.append(df.loc[:, '구간유량보정값'].iloc[i])
    std_week[i-1] = np.std(q)
    if len(q) >= 7*24:
      q = q[1:]
  arrays.append(std_week)

  # diff_std ()
  diff_std = np.zeros([df.shape[0]-1, 1])
  q = []
  for i in range(1, df.shape[0]):
    q.append(df.loc[:, '구간유량보정값'].iloc[i])
    if len(q) <= 7*24:
      diff_std[i-1] = np.std(q)
    else:
      diff_std[i-1] = np.std(q[-7*24:]) - np.std(q[:-7*24]) 
    if len(q) >= 14*24:
      q = q[1:]
  arrays.append(diff_std)

  # MAD
  MAD = np.zeros([df.shape[0]-1, 1])
  q = []
  for i in range(1, df.shape[0]):
    q.append(df.loc[:, '구간유량보정값'].iloc[i])
    tmp_q = np.array(q)
    mid = np.median(tmp_q)
    MAD[i-1] = np.median(np.abs(tmp_q - mid))
    if len(q) >= 7*24:
      q = q[1:]
  arrays.append(MAD)
  
  # 하루동안 총 수신횟수
  cnt = 0
  q = []
  dayreceivecnt = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    cnt += df.loc[:, '수신횟수'].iloc[i]
    q.append(df.loc[:, '수신횟수'].iloc[i])

    dayreceivecnt[i-1] = cnt
    if (len(q) >= 24):
      cnt -= q[0]
      q = q[1:]
  arrays.append(dayreceivecnt)

  # 일주일 총 수신횟수
  cnt = 0
  q = []
  weekreceivecnt = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    cnt += df.loc[:, '수신횟수'].iloc[i]
    q.append(df.loc[:, '수신횟수'].iloc[i])

    weekreceivecnt[i-1] = cnt
    if (len(q) >= 7*24):
      cnt -= q[0]
      q = q[1:]
  arrays.append(weekreceivecnt)

  # 이주일 총 수신횟수
  cnt = 0
  q = []
  weeksreceivecnt = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    cnt += df.loc[:, '수신횟수'].iloc[i]
    q.append(df.loc[:, '수신횟수'].iloc[i])

    weeksreceivecnt[i-1] = cnt
    if (len(q) >= 14*24):
      cnt -= q[0]
      q = q[1:]
  arrays.append(weeksreceivecnt)

  # 한 달 총 수신횟수
  cnt = 0
  q = []
  monthreceivecnt = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    cnt += df.loc[:, '수신횟수'].iloc[i]
    q.append(df.loc[:, '수신횟수'].iloc[i])

    monthreceivecnt[i-1] = cnt
    if (len(q) >= 30*24):
      cnt -= q[0]
      q = q[1:]
  arrays.append(monthreceivecnt)

  # 하루동안 총 수신횟수 (필요량과 차이)
  cnt = 0
  q = []
  dayreceivecnt = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    cnt += df.loc[:, '수신횟수'].iloc[i]
    q.append(df.loc[:, '수신횟수'].iloc[i])

    dayreceivecnt[i-1] = cnt - len(q)
    if (len(q) >= 24):
      cnt -= q[0]
      q = q[1:]
  arrays.append(dayreceivecnt)

  # 일주일 총 수신횟수(필요량과 차이)
  cnt = 0
  q = []
  weekreceivecnt = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    cnt += df.loc[:, '수신횟수'].iloc[i]
    q.append(df.loc[:, '수신횟수'].iloc[i])

    weekreceivecnt[i-1] = cnt - len(q)
    if (len(q) >= 7*24):
      cnt -= q[0]
      q = q[1:]
  arrays.append(weekreceivecnt)

  # 이주일 총 수신횟수 (필요량과 차이)
  cnt = 0
  q = []
  weeksreceivecnt = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    cnt += df.loc[:, '수신횟수'].iloc[i]
    q.append(df.loc[:, '수신횟수'].iloc[i])

    weeksreceivecnt[i-1] = cnt - len(q)
    if (len(q) >= 14*24):
      cnt -= q[0]
      q = q[1:]
  arrays.append(weeksreceivecnt)

  # 한 달 총 수신횟수 (필요량과 차이)
  cnt = 0
  q = []
  monthreceivecnt = np.zeros([df.shape[0]-1, 1])
  for i in range(1, df.shape[0]):
    cnt += df.loc[:, '수신횟수'].iloc[i]
    q.append(df.loc[:, '수신횟수'].iloc[i])

    monthreceivecnt[i-1] = cnt - len(q)
    if (len(q) >= 30*24):
      cnt -= q[0]
      q = q[1:]
  arrays.append(monthreceivecnt)

  # slope (5일전과 오늘의 기울기)
  slope = np.zeros([df.shape[0]-1, 1])
  q = [0 for i in range(5*24-1)]
  for i in range(1, df.shape[0]):
    q.append(df.loc[:, '구간유량보정값'].iloc[i])
    s = sum(q[-24:]) - sum(q[:24])
    if s > 0:
      slope[i-1] = 1
    elif s < 0:
      slope[i-1] = -1
    if len(q) >= 5*24:
      q = q[1:]
  arrays.append(slope)
  
  
  arrays.append(df.loc[:, 'ami배터리'].iloc[1:].to_numpy(np.float, na_value=0).reshape([-1, 1]))
  arrays.append((df.loc[:, 'ami배터리'].iloc[1:] > 0).to_numpy(np.int, na_value=0).reshape([-1, 1]))
  
  tmp_a = np.hstack(arrays)[14*24:]
  tmp_a
  tmp_pd = pd.DataFrame(tmp_a, columns=['label',
                               'flux',
                               'zero_month',
                               'zero_week',
                               'continuous_zero',
                               'diff_weeks',
                               'diff_week_day',
                               'diff_month',
                               'diff_month2',
                               'std_week',
                               'diff_std',
                               'MAD',
                               'rec_day',
                               'rec_week',
                               'rec_2week',
                               'rec_month',
                               'rec_day2',
                               'rec_week2',
                               'rec_2week2',
                               'rec_month2',
                               'slope',
                               'amiBAT',
                               'amiBATQ'])
  tmp_pd = tmp_pd.astype({'label': int, 'zero_month': int,'zero_week': int,'continuous_zero': int, 'amiBATQ': int})
  if total_df is None:
    total_df = tmp_pd
  else:
    total_df = total_df.append(tmp_pd)
  df_cnt += 1

print("="*30)
print(total_df.shape)
print("="*30)
total_df.to_csv("final_data.csv", sep=',', na_rep='NaN')